# Data reduction for Amor

In this notebook, we will look at the reduction workflow for reflectometry data collected from the PSI
[Amor](https://www.psi.ch/en/sinq/amor) instrument.
This is a living document and there are plans to update this as necessary with changes in the data reduction methodology and code.

We will begin by importing the modules that are necessary for this notebook and loading the data.
The `sample.nxs` file is the experimental data file of interest,
while `reference.nxs` is the reference measurement of the neutron supermirror.

In [ ]:
import scipp as sc
import scippneutron as scn
from scippneutron.tof import conversions
from ess import amor, reflectometry as refl

## Loading the data

We first load the `sample.nxs` file and perform some early preprocessing using the `amor.io.load` function.

In [ ]:
sample = amor.load(amor.data.get_path("sample.nxs"))
sample

By simply plotting the data, we get a first glimpse into the data contents.

In [ ]:
sc.plot(sample)

It is also possible to show the instrument view using

In [ ]:
scn.instrument_view(sample)

## Coordinate transformation graph

We begin by defining the convention for naming angles in our set-up.
We use the Fig. 5 from the paper by [Stahn & Glavic (2016)](https://doi.org/10.1016/j.nima.2016.03.007),
which is reproduced below (along with its caption).

![Figure5](https://ars.els-cdn.com/content/image/1-s2.0-S0168900216300250-gr5.jpg)

The yellow area shows the incoming and reflected beam, both with the divergence $\Delta \theta$.
The inclination of the sample relative to the centre of the incoming beam (here identical to the instrument horizon) is called $\omega$,
and the respective angle of the reflected beam relative to the same axis is $\gamma$.
In general the detector centre is located at $\gamma_{\rm D} = 2\omega$.
These are instrument coordinates and should not be confused with the situation on the sample,
where the take-off angle of an individual neutron trajectory is called $\theta$.

To compute the `wavelength`, scattering angles `omega` and `theta`, and the `Q` vector for our data,
we construct a coordinate transformation graph.

It is based on classical conversions from `tof` and pixel `position` to `wavelength`, `two_theta` and `Q`,
but comprises a number of modifications.

The computation of the scattering angle `omega` ignores the $x$ component of the scattered beam ($L_{2}$) vector,
and includes a correction for the Earth's gravitational field which bends the flight path of the neutrons.
The angle can be found using the following
[expression](https://docs.mantidproject.org/v3.9.0/algorithms/Q1D-v2.html#algm-q1d)

$$\omega = \frac{1}{2}\arcsin\left(\frac{\sqrt{x^{2} + \left(y + \frac{g m_{\rm n}}{2 h^{2}} \lambda^{2} L_{2}^{2}\right)^{2}}}{L_{2}}\right)$$

where $m_{\rm n}$ is the neutron mass,
$g$ is the acceleration due to gravity,
and $h$ is Planck's constant
(this assumes the neutrons are all travelling horizontally at the position of the sample).

The conversion graph is defined in the reflectometry module,
and can be obtained via

In [ ]:
graph = refl.conversions.reflectometry_graph()
graph

In [ ]:
sc.show_graph(graph, simplified=True)

## Computing the wavelength

To compute the wavelength of the neutrons,
we request the `wavelength` coordinate from the `transform_coords` method by supplying our graph defined above
(see [here](https://scipp.github.io/scippneutron/user-guide/coordinate-transformations.html)
for more information about using `transform_coords`)

In [ ]:
wavelength = sample.transform_coords(["wavelength"], graph=graph)
wavelength

In [ ]:
wavelength.bins.concatenate('detector_id').plot()

## Compute the Q vector

Using the same method, we can compute the $Q$ vector,
which now depends on both detector position (id) and wavelength

In [ ]:
q_with_gravity = sample.transform_coords(["Q"], graph=graph)
q_with_gravity

In [ ]:
q_edges = sc.linspace(dim='Q', start=0.008, stop=0.08,
                      num=201, unit='1/angstrom')
q_binned_with_g = sc.bin(q_with_gravity, edges=[q_edges])
q_summed_with_g = q_binned_with_g.sum('detector_id')
sc.plot(q_summed_with_g["wavelength", 0], norm="log")

## Normalize by the supermirror

In [ ]:
reference = amor.load(amor.data.get_path("reference.nxs"))
reference

In [ ]:
ref_wavelength = reference.transform_coords(["wavelength"], graph=graph)
ref_wavelength.bins.concatenate('detector_id').plot()

In [ ]:
q_ref = reference.transform_coords(["Q"], graph=graph)
q_ref

In [ ]:
q_ref_binned = sc.bin(q_ref, edges=[q_edges])
q_ref_summed = q_ref_binned.sum('detector_id')
sc.plot(q_ref_summed["wavelength", 0], norm="log")

In [ ]:
sc.plot(q_summed_with_g["wavelength", 0] / q_ref_summed["wavelength", 0], norm="log")

## Compute the $\theta$ angle

In [ ]:
sample_theta = sample.transform_coords(["theta", "wavelength"], graph=graph)
sample_theta

In [ ]:
nbins = 51
theta_edges = sc.linspace(dim='theta', start=0.55, stop=1.3, num=nbins, unit='deg')
wavelength_edges = sc.linspace(dim='wavelength', start=1.0, stop=15.0, num=nbins, unit='angstrom')
binned = sc.bin(sample_theta, edges=[sc.to_unit(theta_edges, 'rad'), wavelength_edges])
binned

In [ ]:
binned.bins.sum().sum('detector_id').plot()